## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
# imports
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from utils import genSankey

# to make notebook work offline
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)


## Data Visualizations

### Permit Data Viz

#### Sankey of permit filing method vs status

In [5]:
# basic sankey
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = ["A1", "A2", "B1", "B2", "C1", "C2"],
      color = "blue"
    ),
    link = dict(
      source = [5, 2, 5, 4, 4, 5], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = [0, 1, 0, 2, 3, 3],
      value = [8, 4, 2, 8, 4, 2]
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()

In [2]:
df_transfers = pd.read_csv('data\processed_data\Parcel_Transfers.csv')
df_transfers_residential = df_transfers.loc[df_transfers['DevelopmentRight']=='Coverage (hard)']
df_transfers_residential_sending = df_transfers_residential.loc[df_transfers_residential['SendingVsReceiving']=='Sending']


In [3]:
df_transfers_grouped = df_transfers_residential_sending.groupby(['LandCapabilityCategory','CounterpartSensitivity']).agg({'CumulativeBankedQuantity':'sum'}).reset_index()

In [4]:
# where LandCapabilityCategory is equal to 'Non-Sensitive' set it equal to NonSensitive sending
df_transfers_grouped.loc[df_transfers_grouped['LandCapabilityCategory']=='Non-Sensitive','LandCapabilityCategory'] = 'NonSensitive sending'
# where LandCapabilityCategory is equal to 'Sensitive' set it equal to Sensitive sending
df_transfers_grouped.loc[df_transfers_grouped['LandCapabilityCategory']=='Sensitive','LandCapabilityCategory'] = 'Sensitive sending'

In [5]:
residential = genSankey(df_transfers_grouped, category_cols=['LandCapabilityCategory','CounterpartSensitivity'], value_cols='CumulativeBankedQuantity', title='Residential Transfers')

In [6]:
fig = go.Figure(residential)
iplot(fig)

In [10]:
fig.write_html(r'C:\Users\amcclary\Documents\GitHub\Reporting\html\residential_sankey.html')

In [5]:
df = pd.read_csv('permit_demo.csv')
df.head()

,Permit_Type,Permit_Submittal_Method,Permit_Outcome,Total_Permits,Total_Dummy_Column
0,Littoral,Electronic,Approved,50,Total
1,Littoral,In Person,Approved,20,Total
2,Littoral,Other,Approved,15,Total
3,Littoral,Electronic,Denied,5,Total
4,Littoral,In Person,Denied,3,Total


In [7]:
# Generating DFs for different filter options
Littoral = genSankey(df[df['Permit_Type']=='Littoral'],category_cols=['Permit_Submittal_Method', 'Permit_Type','Permit_Outcome'],value_cols='Total_Permits',title='Permits?')
Development = genSankey(df[df['Permit_Type']=='Development'],category_cols=['Permit_Submittal_Method','Permit_Type','Permit_Outcome'],value_cols='Total_Permits',title='Permits?')
all = genSankey(df,category_cols=['Permit_Submittal_Method','Permit_Type','Permit_Outcome'],value_cols='Total_Permits',title='Permits?')

# Constructing menus
updatemenus = [{'buttons': [{'method': 'animate',
                             'label': 'All',
                             'args': [all]
                              },
                            {'method': 'animate',
                             'label': 'Littoral',
                             'args': [Littoral]
                             },
                            {'method': 'animate',
                             'label': 'Development',
                             'args': [Development]
                             }
                            ] } ]

# update layout with buttons, and show the figure
sank = genSankey(df,category_cols=['Permit_Submittal_Method','Permit_Type','Permit_Outcome'],value_cols='Total_Permits',title='Permits?')
fig = go.Figure(sank)
fig.update_layout(updatemenus=updatemenus)
iplot(fig)
fig.write_html('demo.html')